In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from datetime import timedelta

from scipy.spatial import cKDTree

In [2]:
pdf = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\df_preprocessed.csv")

In [3]:
cesmATom1 = xr.open_dataset(r"D:\PVOCAL\CESM\CESM-FINN_DC8_20160729_R0.nc")
cesmATom2 = xr.open_dataset(r"D:\PVOCAL\CESM\CESM-FINN_DC8_20170126_R0.nc")
cesmATom3 = xr.open_dataset(r"D:\PVOCAL\CESM\CESM-FINN_DC8_20170928_R0.nc")
cesmATom4 = xr.open_dataset(r"D:\PVOCAL\CESM\CESM-FINN_DC8_20180424_R0.nc")

In [ ]:
cesmATom1.data_vars

Data variables:
    lon             (time) float32 ...
    T_CESM          (time) float32 ...
    U_CESM          (time) float32 ...
    V_CESM          (time) float32 ...
    O3S_CESM        (time) float32 ...
    Z3_CESM         (time) float32 ...
    OMEGA_CESM      (time) float32 ...
    PM25_CESM       (time) float32 ...
    CO_CESM         (time) float32 ...
    E90_CESM        (time) float32 ...
    BR_CESM         (time) float32 ...
    BRCL_CESM       (time) float32 ...
    BRO_CESM        (time) float32 ...
    BROX_CESM       (time) float32 ...
    BROY_CESM       (time) float32 ...
    TBRY_CESM       (time) float32 ...
    BRONO2_CESM     (time) float32 ...
    CCL4_CESM       (time) float32 ...
    CF2CLBR_CESM    (time) float32 ...
    CF3BR_CESM      (time) float32 ...
    CFC11_CESM      (time) float32 ...
    CFC113_CESM     (time) float32 ...
    CFC12_CESM      (time) float32 ...
    CH2O_CESM       (time) float32 ...
    CH3BR_CESM      (time) float32 ...
    CHBR3

In [5]:
cesm_atom_1 = cesmATom1.to_dataframe()
cesm_atom_1 = cesm_atom_1.reset_index()
cesm_atom_1.rename(columns={"obs_time": "Datetime"}, inplace=True)
cesm_atom_2 = cesmATom2.to_dataframe()
cesm_atom_2 = cesm_atom_2.reset_index()
cesm_atom_2.rename(columns={"obs_time": "Datetime"}, inplace=True)
cesm_atom_3 = cesmATom3.to_dataframe()
cesm_atom_3 = cesm_atom_3.reset_index()
cesm_atom_3.rename(columns={"obs_time": "Datetime"}, inplace=True)
cesm_atom_4 = cesmATom4.to_dataframe()
cesm_atom_4 = cesm_atom_4.reset_index()
cesm_atom_4.rename(columns={"obs_time": "Datetime"}, inplace=True)

cesm_atom = pd.concat([cesm_atom_1, cesm_atom_2, cesm_atom_3, cesm_atom_4], ignore_index=True)

# Optional: Sort by 'Datetime' if needed
cesm_atom.sort_values(by='Datetime', inplace=True)

# # Save the combined DataFrame to a CSV
# output_path = r'C:\Users\vwgei\Documents\PVOCAL\data\V4\cesm_atom.csv'
# cesm_atom.to_csv(output_path, index=False)

In [6]:
# Already Done
atom_gmi = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\atom_gmi.csv")

winter = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\WINTER.csv")

# Define a custom parser function
def parse_datetime(date, hour, minute):
    # Convert the DATE column (in the format YYYYMMDD) to datetime and combine with HOUR and MINUTE
    return pd.to_datetime(str(date) + f'{int(hour):02d}{int(minute):02d}', format='%Y%m%d%H%M')

# join on date, hour, minute # ppbv # should have a Datetime
intexb_c130_geos_chem = pd.read_csv(r"D:\PVOCAL\model_data\intexb_c130_GEOS-Chem.csv")
# Create a new 'Datetime' column by combining 'DATE', 'HOUR', and 'MINUTE'
intexb_c130_geos_chem['Datetime'] = pd.to_datetime(
    intexb_c130_geos_chem['DATE'].astype(str) + 
    intexb_c130_geos_chem['HOUR'].astype(int).apply(lambda x: f'{x:02d}') + 
    intexb_c130_geos_chem['MINUTE'].astype(int).apply(lambda x: f'{x:02d}'), 
    format='%Y%m%d%H%M'
)
intexb_dc8_geos_chem = pd.read_csv(r"D:\PVOCAL\model_data\intexb_dc8_GEOS-Chem.csv")
# Create a new 'Datetime' column by combining 'DATE', 'HOUR', and 'MINUTE'
intexb_dc8_geos_chem['Datetime'] = pd.to_datetime(
    intexb_dc8_geos_chem['DATE'].astype(str) + 
    intexb_dc8_geos_chem['HOUR'].astype(int).apply(lambda x: f'{x:02d}') + 
    intexb_dc8_geos_chem['MINUTE'].astype(int).apply(lambda x: f'{x:02d}'), 
    format='%Y%m%d%H%M'
)

# Now have a Datetime # MAKE SURE TO CONVERT UNITS
intexb_c130_raqms = pd.read_csv(r"D:\PVOCAL\model_data\intexb_c130_RAQMS.csv")
intexb_dc8_raqms = pd.read_csv(r"D:\PVOCAL\model_data\intexb_dc8_RAQMS.csv")

# lat lon alt # ppbv # "DATE" yyymmdd also "UTC" # ALT in km
intexna_geos_chem = pd.read_csv(r"D:\PVOCAL\model_data\intexna_GEOS-Chem.csv")
# Convert the 'DATE' column to datetime (assuming format 'yyyymmdd')
intexna_geos_chem['DATE'] = pd.to_datetime(intexna_geos_chem['DATE'], format='%Y%m%d')
# Convert 'UTC' fractional hours to timedelta (convert fractional hours to seconds)
intexna_geos_chem['UTC_seconds'] = intexna_geos_chem['UTC'] * 3600
# Add the 'UTC_seconds' to the 'DATE' column to get the 'Datetime' column
intexna_geos_chem['Datetime'] = intexna_geos_chem['DATE'] + pd.to_timedelta(intexna_geos_chem['UTC_seconds'], unit='s')

# Now has a Datetime column
intexna_raqms = pd.read_csv(r"D:\PVOCAL\model_data\intexna_GEOS-Chem.csv")

# Datetime
korusaq_micp = pd.read_csv(r"D:\PVOCAL\model_data\korusaq_MICP.csv")
korusaq_micp['Datetime'] = pd.to_datetime(korusaq_micp['Datetime'], format='%m/%d/%Y %H:%M')
# vmrc2h6_GEOSChemSNU, ppbv
# vmrc2h6_CAMChemNCAR, ppbv
# vmrc2h6_WRFChemNCAR, ppb
# vmrc2h6_WRFChemPNU, ppbv
# c2h6_WRFChemUCLA, ppbv

pta_aer_point_model = pd.read_csv(r"D:\PVOCAL\model_data\pta_AER-point-model.csv")
pta_aer_point_model.columns = pta_aer_point_model.columns.str.replace(',', '')
# Assume the year is 1996
base_date = pd.to_datetime('1996-01-01')

# Calculate the Datetime by adding days (Julian day) and seconds (from 'time')
pta_aer_point_model['Datetime'] = base_date + pd.to_timedelta(pta_aer_point_model['date'] - 1, unit='D') + pd.to_timedelta(pta_aer_point_model['time'], unit='s')

# Datetime
sec4rs_geos_chem = pd.read_csv(r"D:\PVOCAL\model_data\sec4rs_GEOS-Chem.csv")
sec4rs_geos_chem['Datetime'] = pd.to_datetime(
    sec4rs_geos_chem['DATE'].astype(str) + 
    sec4rs_geos_chem['HOUR'].astype(int).apply(lambda x: f'{x:02d}') + 
    sec4rs_geos_chem['MINUTE'].astype(int).apply(lambda x: f'{x:02d}'), 
    format='%Y%m%d%H%M'
)


# Assume the year is 2001
base_date = pd.to_datetime('2001-01-01')

tracep_dc8 = pd.read_csv(r"D:\PVOCAL\model_data\tracep_DC8.csv")
# Calculate the Datetime by adding days and seconds
tracep_dc8['Datetime'] = base_date + pd.to_timedelta(tracep_dc8['Day'] - 1, unit='D') + pd.to_timedelta(tracep_dc8['GMT'], unit='s')

tracep_p3 = pd.read_csv(r"D:\PVOCAL\model_data\tracep_P3.csv")
# Calculate the Datetime by adding days and seconds
tracep_p3['Datetime'] = base_date + pd.to_timedelta(tracep_p3['Day'] - 1, unit='D') + pd.to_timedelta(tracep_p3['GMT'], unit='s')


# All should now have a Datetime

In [7]:
# ACCLIP = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\ACCLIP.csv", parse_dates=['Datetime'])
# DC3 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\DC3.csv")
# DISCOVERAQ = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\DISCOVERAQ_FRAPPE.csv")
# FIREXAQ = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\FIREXAQ.csv", parse_dates=['Datetime'])
KORUSAQ = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\KORUSAQ.csv")
SECA4RS_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\SEAC4RS_DC8.csv")
WINTER = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\WINTER.csv")

INTEXA = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\INTEXA.csv")
INTEXB_C130 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\INTEXB_C130.csv")
INTEXB_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\INTEXB_DC8.csv")
PEMTA_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMTA_DC8.csv")
PEMTA_P3 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMTA_P3.csv")
# PEMTB_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMTB_DC8.csv")
# PEMTB_P3 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMTB_P3.csv")
# PEMWA = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMWA.csv")
# PEMWB = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\PEMWB.csv")
# TRACEA_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\TRACEA_DC8.csv")
# TRACEA_P3 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\TRACEA_P3.csv")
TRACEP_DC8 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\TRACEP_DC8.csv")
TRACEP_P3 = pd.read_csv(r"D:\PVOCAL\Other_Campaigns\TRACEP_P3.csv")

ATOM = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\old\A_NEW_ERA5_PASTEL_backup_full.csv")

In [8]:
def clean_datetime(row):
    # Base datetime from Year, Month, Day, and Hour
    base_datetime = pd.Timestamp(year=row["Year"], month=row["Month"], day=row["Day"], hour=row["Hour"])

    # Case 1: Good datetime format
    if "/" in row["Datetime"]:  # e.g., '7/29/2016 14:45'
        try:
            parsed_datetime = pd.to_datetime(row["Datetime"], format="%m/%d/%Y %H:%M")
            return parsed_datetime
        except ValueError:
            pass  # Fallback to case 2 if parsing fails

    # Case 2: "Half" datetime (MM:SS)
    if ":" in row["Datetime"]:  # e.g., '26:29.5'
        try:
            mm_ss = row["Datetime"].split(":")  # Split into MM and SS
            minutes = int(mm_ss[0])
            seconds = float(mm_ss[1]) if len(mm_ss) > 1 else 0
            adjusted_datetime = base_datetime + timedelta(minutes=minutes, seconds=seconds)
            return adjusted_datetime
        except Exception as e:
            print(f"Error processing row {row.name}: {e}")
            return None

    # Return None if no valid datetime was found
    return None

In [9]:
# List of all DataFrames
dataframes = [
    KORUSAQ, SECA4RS_DC8, WINTER,
    INTEXA, INTEXB_C130, INTEXB_DC8, PEMTA_DC8, PEMTA_P3, 
    TRACEP_DC8, TRACEP_P3
]

# Apply clean_datetime function and rename columns
for i, df in enumerate(dataframes):
    try:
        # Apply the function row-wise and store results in a new column
        df["Cleaned_Datetime"] = df.apply(clean_datetime, axis=1)
        
        # Rename the old "Datetime" column to "Datetime_old" and the cleaned column to "Datetime"
        if "Datetime" in df.columns:
            df.rename(columns={"Datetime": "Datetime_old", "Cleaned_Datetime": "Datetime"}, inplace=True)
        else:
            df.rename(columns={"Cleaned_Datetime": "Datetime"}, inplace=True)  # Handle case if "Datetime" doesn't exist

        print(f"Cleaned and renamed columns for DataFrame {i + 1} successfully!")
    except Exception as e:
        print(f"Error processing DataFrame {i + 1}: {e}")

Cleaned and renamed columns for DataFrame 1 successfully!
Cleaned and renamed columns for DataFrame 2 successfully!
Cleaned and renamed columns for DataFrame 3 successfully!
Cleaned and renamed columns for DataFrame 4 successfully!
Cleaned and renamed columns for DataFrame 5 successfully!
Cleaned and renamed columns for DataFrame 6 successfully!
Cleaned and renamed columns for DataFrame 7 successfully!
Cleaned and renamed columns for DataFrame 8 successfully!
Cleaned and renamed columns for DataFrame 9 successfully!
Cleaned and renamed columns for DataFrame 10 successfully!


In [10]:
# ATOM <- atom_gmi, cesm_atom
# SECA4RS_DC8 <- sec4rs_geos_chem
# INTEXA <- intexna_geos_chem, intexna_raqms
# INTEXB_C130 <- intexb_c130_geos_chem, intexb_c130_raqms
# INTEXB_DC8 <- intexb_dc8_geos_chem,intexb_dc8_raqms
# PEMTA_DC8 <- pta_aer_point_model
# PEMTA_P3 <- pta_aer_point_model
# TRACEP_DC8 <- tracep_dc8
# TRACEP_P3 <- tracep_p3
# KORUSAQ <- korusaq_micp

In [11]:
KORUSAQ['Datetime'] = pd.to_datetime(KORUSAQ['Datetime'], format='%m/%d/%Y %H:%M')

In [12]:
# KORUSAQ['Datetime'] = pd.to_datetime(KORUSAQ['Datetime'], format='%m/%d/%Y %H:%M')

KORUSAQ = KORUSAQ.sort_values('Datetime').reset_index(drop=True)
korusaq_micp = korusaq_micp.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
korusaq_micp_f = pd.merge_asof(
    KORUSAQ,
    korusaq_micp,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs", "_MICP")
)

In [13]:
# korusaq_micp_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\korusaq_micp.csv", index=False)

In [14]:
TRACEP_P3['Datetime'] = pd.to_datetime(TRACEP_P3['Datetime'], format='%m/%d/%Y %H:%M')

TRACEP_P3 = TRACEP_P3.sort_values('Datetime').reset_index(drop=True)
tracep_p3 = tracep_p3.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
tracep_p3_f = pd.merge_asof(
    TRACEP_P3,
    tracep_p3,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_box-model")
)

In [15]:
TRACEP_DC8['Datetime'] = pd.to_datetime(TRACEP_DC8['Datetime'], format='%m/%d/%Y %H:%M')

TRACEP_DC8 = TRACEP_DC8.sort_values('Datetime').reset_index(drop=True)
tracep_dc8 = tracep_dc8.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
tracep_dc8_f = pd.merge_asof(
    TRACEP_DC8,
    tracep_dc8,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_box-model")
)

In [16]:
for c in PEMTA_P3.columns:
    print(c)

iindex
JDAY
STARTTIME
STOPTIME
MIDTIME
LAT
wLAT
LON
wLON
PSMB
wPSMB
ALT
wMALTP
TS
wTS
TD
wTD
PITCH
wPITCH
ROLL
wROLL
MTAS
wMTAS
MGS
wMGS
MACH
wMACH
SURTEMPK
wSURTEMPK
POTTEMP
wPOTTEMP
calcPOTTEMP
wcalcPOTTEMP
SH1011
wSH1011
RHw
wRHw
RHi
wRHi
VPPW
wVPPW
VPPI
wVPPI
VPSW
wVPSW
VPSI
wVPSI
WNS
wMWNS
WND
wWND
UVZ
wUVZ
UVN
wUVN
O3CLM
wO3CLM
CH3ONO2_A
wCH3ONO2_A
C2H5ONO2_A
wC2H5ONO2_A
i-C3H7ONO2_A
wi-C3H7ONO2_A
n-C3H7ONO2_A
wn-C3H7ONO2_A
2-C4H9ONO2_A
w2-C4H9ONO2_A
3-MC4H9ONO2_A
w3-MC4H9ONO2_A
3-C5H11ONO2_A
w3-C5H11ONO2_A
2-C5H11ONO2_A
w2-C5H11ONO2_A
CH2Cl2_A
wCH2Cl2_A
HCFC141B_A
wHCFC141B_A
HCFC142B_A
wHCFC142B_A
HFC134A_A
wHFC134A_A
CH2Br2_A
wCH2Br2_A
CHBrCl2_A
wCHBrCl2_A
CHBr2Cl_A
wCHBr2Cl_A
CHBr3_A
wCHBr3_A
SO2
wSO2
DMS_WAS
wDMS
H-1301_B
wH-1301_B
H-1211_B
wH-1211_B
H-2402_B
wH-2402_B
HCFC-22_B
wHCFC-22_B
F12_B
wF12_B
F11_B
wF11_B
F113_B
wF113_B
F114_B
wF114_B
CH3Cl_B
wCH3Cl_B
CH3Br_WAS
wCH3Br_B
CH3I_B
wCH3I_B
CHCl3_B
wCHCl3_B
CH3CCl3_B
wCH3CCl3_B
CCl4_B
wCCl4_B
C2Cl4_B
wC2Cl4_B
CHBrCl2_B
w

In [17]:
for c in pta_aer_point_model.columns:
    print(c)

flight
date
time
lat
long
sza
altitude
temp
pres
O1D
O
O21
O3
H
OH
HO2
H2O2
CH3O2
CH3O2NO2
CH3OOH
CH2O
NO
NO2
HNO3
NO3
N2O5
HO2NO2
HONO
PAN
PPN
C2H5O2
C2H5OOH
CH3CHO
CH3CO3
CH3CO3H
C2H2O2NO2
ALKA
C4H10
C2H4
ISOPRENE
H2O
CH4
C2H6
C3H8
ACET
CO
obsOH
obsHO2
obsNO2
O3
NetP
HNO
H2O2+hv->OH+OH
HO2NO2+hv->OH+NO3
CH3OOH+hv->CH2O+OH+HO2C2H5OOH+hv->CH3CHO+HO2+OH
CH3CO3H+hv->CH2O+OH+HO2
H2O+O(1D)->OH+OH
CH4+O(1D)->CH3O2+OH
H2+O(1D)->OH+H
HO2+O3->OH+2*O2
HO2+NO->OH+NO2
CH3OOH+OH->CH2O+OH+H2OC2H5OOH+OH->CH3CHO+OH+H2O
CH3CO3H+OH->CH2O+OH+CO
OH+O3->HO2+O2
OH+H2O2->H2O+HO2
HO2+OH->H2O+O2
OH+CO->H+XX
OH+CH4->CH3O2
OH+NO2+M->HNO3+M
OH+HNO3->H2O+NO3
OH+OH->H2O+O
OH+CH2O->H2O+CO+H
OH+HO2NO2->H2O+NO2
OH+C2H6+M->C2H5O2+M
OH+CH3OOH->CH3O2+H2O
OH+CH3OOH->CH2O+OH+H2O
OH+C2H5OOH->C2H5O2+H2OOH+C2H5OOH->CH3CHO+OH+H2O
OH+CH3CHO->CH3CO3+H2O
OH+PAN->CH2O+NO2+H2O
OH+CH3CO3H->CH3CO3+H2O
OH+CH3CO3H->CH2O+OH+CO
CH2O+hv->2*H+CO
HO2NO2+hv->OH+NO3.1
CH3CHO+hv->CH3O2+CO+HO2
OH+H2O2->H2O+HO2.1
HO2NO2->HO2+NO2
CH3O3+NO->CH2O+

In [18]:
pta_aer_point_model['ALT'] = pta_aer_point_model['altitude'] * 1000
print(pta_aer_point_model['ALT'].min())
print(PEMTA_P3['ALT'].min())


156.0
-999990


In [19]:
PEMTA_P3['Datetime'] = pd.to_datetime(PEMTA_P3['Datetime'], format='%m/%d/%Y %H:%M')

PEMTA_P3 = PEMTA_P3.sort_values('Datetime').reset_index(drop=True)
pta_aer_point_model = pta_aer_point_model.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
pta_aer_point_model_p3_f = pd.merge_asof(
    PEMTA_P3,
    pta_aer_point_model,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_AER-point-model")
)

In [20]:
PEMTA_DC8['Datetime'] = pd.to_datetime(PEMTA_DC8['Datetime'], format='%m/%d/%Y %H:%M')

PEMTA_DC8 = PEMTA_DC8.sort_values('Datetime').reset_index(drop=True)
pta_aer_point_model = pta_aer_point_model.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
pta_aer_point_model_dc8_f = pd.merge_asof(
    PEMTA_DC8,
    pta_aer_point_model,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_AER-point-model")
)

In [21]:
INTEXB_DC8['Datetime'] = pd.to_datetime(INTEXB_DC8['Datetime'], format='%m/%d/%Y %H:%M')
intexb_dc8_raqms['Datetime'] = pd.to_datetime(intexb_dc8_raqms['Datetime'], format='%m/%d/%Y %H:%M')

INTEXB_DC8 = INTEXB_DC8.sort_values('Datetime').reset_index(drop=True)
intexb_dc8_raqms = intexb_dc8_raqms.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
intexb_dc8_raqms_f = pd.merge_asof(
    INTEXB_DC8,
    intexb_dc8_raqms,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_RAQMS")
)

In [22]:
INTEXB_DC8['Datetime'] = pd.to_datetime(INTEXB_DC8['Datetime'], format='%m/%d/%Y %H:%M')
intexb_dc8_geos_chem['Datetime'] = pd.to_datetime(intexb_dc8_geos_chem['Datetime'], format='%m/%d/%Y %H:%M')

INTEXB_DC8 = INTEXB_DC8.sort_values('Datetime').reset_index(drop=True)
intexb_dc8_geos_chem = intexb_dc8_geos_chem.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
intexb_dc8_geos_chem_f = pd.merge_asof(
    INTEXB_DC8,
    intexb_dc8_geos_chem,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_geos-chem")
)

In [23]:
INTEXB_C130['Datetime'] = pd.to_datetime(INTEXB_C130['Datetime'], errors='coerce', infer_datetime_format=True)
intexb_c130_raqms['Datetime'] = pd.to_datetime(intexb_c130_raqms['Datetime'], errors='coerce', infer_datetime_format=True)

# Ensure both DataFrames are sorted by Datetime
INTEXB_C130 = INTEXB_C130.sort_values('Datetime').reset_index(drop=True)
intexb_c130_raqms = intexb_c130_raqms.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
intexb_c130_raqms_f = pd.merge_asof(
    INTEXB_C130,
    intexb_c130_raqms,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_RAQMS")
)

C:\Users\vwgei\AppData\Local\Temp\ipykernel_7084\1328419973.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  INTEXB_C130['Datetime'] = pd.to_datetime(INTEXB_C130['Datetime'], errors='coerce', infer_datetime_format=True)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_7084\1328419973.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  intexb_c130_raqms['Datetime'] = pd.to_datetime(intexb_c130_raqms['Datetime'], errors='coerce', infer_datetime_format=True)


In [24]:
INTEXB_C130['Datetime'] = pd.to_datetime(INTEXB_C130['Datetime'], format='%m/%d/%Y %H:%M')

INTEXB_C130 = INTEXB_C130.sort_values('Datetime').reset_index(drop=True)
intexb_c130_geos_chem = intexb_c130_geos_chem.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
intexb_c130_geos_chem_f = pd.merge_asof(
    INTEXB_C130,
    intexb_c130_geos_chem,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_geos-chem")
)

In [25]:
INTEXA['Datetime'] = pd.to_datetime(INTEXA['Datetime'], format='%m/%d/%Y %H:%M')
INTEXA.dropna(subset=['Datetime'], inplace=True)

INTEXA = INTEXA.sort_values('Datetime').reset_index(drop=True)
intexna_geos_chem = intexna_geos_chem.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
intexna_geos_chem_f = pd.merge_asof(
    INTEXA,
    intexna_geos_chem,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_geos-chem")
)

In [26]:
SECA4RS_DC8['Datetime'] = pd.to_datetime(SECA4RS_DC8['Datetime'], format='%m/%d/%Y %H:%M')

SECA4RS_DC8 = SECA4RS_DC8.sort_values('Datetime').reset_index(drop=True)
sec4rs_geos_chem = sec4rs_geos_chem.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
seac4rs_f = pd.merge_asof(
    SECA4RS_DC8,
    sec4rs_geos_chem,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_geos-chem")
)

In [27]:
# ATOM['Datetime'].to_datetime(format='%Y-%m-%d %H:%M:00')
ATOM['Datetime'] = pd.to_datetime(ATOM['Datetime'], format='%m/%d/%Y %H:%M')

ATOM = ATOM.sort_values('Datetime').reset_index(drop=True)
cesm_atom = cesm_atom.sort_values('Datetime').reset_index(drop=True)

# Perform the asof merge
atom_f = pd.merge_asof(
    ATOM,
    cesm_atom,
    on="Datetime",  # Key column for the nearest match
    direction="backward",  # Matches the closest time in either direction
    suffixes=("_obs","_cesm")
)

In [28]:
final_dfs = [atom_f, atom_gmi, seac4rs_f, intexna_geos_chem_f,
             intexb_c130_geos_chem_f, intexb_c130_raqms_f,
             intexb_dc8_geos_chem_f, intexb_dc8_raqms_f,
             pta_aer_point_model_dc8_f, pta_aer_point_model_p3_f,
             tracep_dc8_f, tracep_p3_f, korusaq_micp_f, winter]

# Check em off here:
#atom_f, seac4rs_f, intexna_geos_chem_f, intexb_c130_geos_chem_f, intexb_c130_raqms_f
#intexb_dc8_geos_chem_f, intexb_dc8_raqms_f, pta_aer_point_model_dc8_f, pta_aer_point_model_p3_f
# tracep_dc8_f, tracep_p3_f, korusaq_micp_f

counter = 0
for df in final_dfs:
    # Replace WAS nodata value with np.nan for consistency
    df.replace(-999999.0, np.nan, inplace=True)
    df.replace(-888888.0, np.nan, inplace=True)
    df.replace(-888888888.0, np.nan, inplace=True)
    df.replace(-999999999.0, np.nan, inplace=True)
    df.replace(-99999.000000, np.nan, inplace=True)
    df.replace(-999.990000, np.nan, inplace=True)
    df.replace(-9.999999e+09, np.nan, inplace=True)
    df.replace(-9.999990e+08, np.nan, inplace=True)
    df.replace(-8.888888e+06, np.nan, inplace=True)
    df.replace(-9.999999e+06, np.nan, inplace=True)

    # Replace WAS nodata value with np.nan for consistency
    df.replace(-8888, np.nan, inplace=True)
    df.replace(-999, np.nan, inplace=True)
    df.replace(-888, np.nan, inplace=True)
    df.replace(-777, np.nan, inplace=True)
    df.replace(-777.770000, np.nan, inplace=True)
    df.replace(-888.800000, np.nan, inplace=True)
    df.replace(-888.880000, np.nan, inplace=True)
    df.replace(10000000, np.nan, inplace=True)

    # Replace values in df['DMS_WAS'] lower than 1 with np.nan
    # try:
    #     df.loc[df['DMS'] < 1, 'DMS'] = np.nan

    # except:
    #     try:
    #         df.loc[df['DMS_WAS'] < 1, 'DMS_WAS'] = np.nan
    #     except KeyError:
    #         print(f"df {counter} doesn't have dms... egg")

    # try:
    #     df.loc[df['O3'] < 1, 'O3'] = np.nan
    # except:
    #     try:
    #         df.loc[df['O3_obs'] < 1, 'O3'] = np.nan
    #     except KeyError:
    #         print(f"df {counter} doesn't have O3... egg")

    # try:
    #     df.loc[df['CH4'] < 1500, 'CH4'] = np.nan
    # except:
    #     try:
    #         df.loc[df['CH4_obs'] < 1500, 'CH4'] = np.nan
    #     except KeyError:
    #         print(f"df {counter} doesn't have CH4... egg")

    counter += 1

In [29]:
# atom_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\atom_cesm.csv", index=False)
# atom_gmi.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\atom_gmi.csv", index=False)
# seac4rs_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\seac4rs_geos-chem.csv", index=False)
# intexna_geos_chem_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\intexna_geos-chem.csv", index=False)
# winter.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\winter_geos-chem.csv", index=False)
# intexb_c130_geos_chem_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\intexb_c130_geos-chem.csv", index=False)
# intexb_c130_raqms_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\intexb_c130_raqms.csv", index=False)
# intexb_dc8_geos_chem_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\intexb_dc8_geos-chem.csv", index=False)
# intexb_dc8_raqms_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\intexb_dc8_raqms.csv", index=False)


In [241]:
from sklearn.metrics import r2_score
from sklearn.metrics import d2_absolute_error_score
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
import os


# Define a function that calcualte error metrics from predicted and actual values
def reg_model_metrics(actual,pred):
    MSE = mean_squared_error(actual,pred)
    RMSE = np.sqrt(MSE)
    actual_mean = np.mean(actual)
    RRMSE = 100*RMSE/actual_mean
    MAE = mean_absolute_error(actual, pred)
    R2 = r2_score(actual,pred)
    D2 = d2_absolute_error_score(actual, pred)
    MAXErr = max_error(actual, pred)
    EVS = explained_variance_score(actual, pred)
    return MSE, RMSE, RRMSE, MAE, R2, D2, MAXErr, EVS 

def scatter_plot(actual, pred, title, var_of_int, unit_label, comp_flag, save_path):
    # Color Matching for clarity
    if var_of_int == 'CH4':
        color = 'blue'
        cm = 'Blues'
    elif var_of_int == 'DMS':
        color = 'purple'
        cm = 'Purples'
    elif var_of_int == 'CO':
        color = 'cyan'
        cm = 'YlGnBu'
    elif var_of_int == 'O3':
        color = 'gold'
        cm = 'YlOrBr'
    elif var_of_int == 'Benzene':
        color = 'red'
        cm = 'Reds'
    elif var_of_int == 'CH3Br':
        color = 'mediumseagreen'
        cm = 'Greens'
    elif var_of_int == 'Ethane':
        color = 'orange'
        cm = 'Oranges'
    else:
        color = 'gray'
        cm = 'Greys'
        
    if comp_flag:
        cm = "Greys"
    
    MSE, RMSE, RRMSE, MAE, R2, D2, MAXErr, EVS = reg_model_metrics(actual, pred)
        
    fig,ax = plt.subplots(figsize=(8, 6))
    ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
    ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
    text = r"EVS = %.2f" % (EVS); text += "\n";
    text += r"MAE = %.2f" % (MAE); text += "\n";
    text += r"MSE = %.2f" % (MSE); text += "\n";
    text += r"RMSE = %.2f" % (RMSE); text += "\n";     
    text += r"RRMSE = %.2f" % (RRMSE); text += "\n";
    plt.annotate(text, xy=(0.01, 0.9), xycoords='axes fraction',color='black', fontsize=10,bbox=dict(facecolor='none', edgecolor='none'))
    ax.set_xlabel(f'Measured {var_of_int} {unit_label}')
    ax.set_ylabel(f'Predicted {var_of_int} {unit_label}')
    # # Set log scale for x and y axes
    ax.set_xscale('log')
    ax.set_yscale('log')
    # Create locators to specify the ticks on both axes
    ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))  # Controls major ticks
    ax.xaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs="auto"))  # Controls minor ticks

    ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs="auto"))

    # Ensure minor gridlines are visible and major gridlines are configured
    ax.grid(visible=True, which="major", linestyle="--", linewidth=0.5)  # Major gridlines
    ax.grid(visible=True, which="minor", linestyle=":", linewidth=0.5)  # Minor gridlines
    ax.set_title(title)
    # plt.grid()
    plt.savefig(save_path)
    # plt.show()
    plt.clf()
    return None

In [38]:
dms_cesm = atom_f['DMS_CESM'] * 1e12
dms_cesm.max()

1728.5504

In [60]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\atom_cesm"

# Define the metadata for each variable
variables = [
    {"name": "CH4", "cesm_name": "CH4_CESM", "factor": 1e9, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "cesm_name": "O3_CESM", "factor": 1e9, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "cesm_name": "CO_CESM", "factor": 1e9, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    {"name": "CH3Br", "cesm_name": "CH3BR_CESM", "factor": 1e12, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane", "cesm_name": "C2H6_CESM", "factor": 1e12, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS", "cesm_name": "DMS_CESM", "factor": 1e12, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = atom_f.dropna(subset=[var["name"], var["cesm_name"]])

    # Additional filtering for DMS_CESM with scaling before filtering
    if var["cesm_name"] == "DMS_CESM":
        data = data[data[var["cesm_name"]] * var["factor"] >= 1]

    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["cesm_name"]] * var["factor"]

    # Generate the title and file path
    title = f'CESM Modeled {var["label"].lower()} for ATom'
    file_path = os.path.join(save_path, f"atom_cesm_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provide

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [59]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\atom_gmi"

os.makedirs(save_path, exist_ok=True)

campaign = "ATom"
name_of_model = "GMI" 

# Define the metadata for each variable
variables = [
    {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "model_name": "GMI_O3_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "model_name": "GMI_CO_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    {"name": "CH3Br_WAS", "model_name": "GMI_CH3Br_GMI", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "GMI_Ethane_GMI", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    # {"name": "DMS_WAS", "model_name": "DMS_CESM", "factor": 1e11, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = atom_gmi.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE
    
    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provide

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [61]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\seac4rs_geos-chem"

os.makedirs(save_path, exist_ok=True)

campaign = "SEAC4RS"
name_of_model = "GEOS-Chem" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3_obs", "model_name": "O3_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO_obs", "model_name": "CO_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # most_frequent_dms = seac4rs_f['DMS'].value_counts().idxmax()
    # Drop rows where 'DMS' equals the most frequent value
    # seac4rs_f = seac4rs_f[seac4rs_f['DMS'] != most_frequent_dms]

    # Filter the data for non-missing values
    data = seac4rs_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE
    
    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provide

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [245]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\intexna_geos-chem"

os.makedirs(save_path, exist_ok=True)

campaign = "INTEXNA"
name_of_model = "GEOS-Chem" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    # {"name": "O3_obs", "model_name": "O3_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO_obs", "model_name": "CO_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = intexna_geos_chem_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]

    mask = data[var["name"]] > 0
    data = data[mask]

    mask = data[var["model_name"]] > 0
    data = data[mask]

    if var['name'] == "DMS_WAS":
        mask1 = (data[var["model_name"]] * var["factor"]) < 10000
        data = data[mask1]

    if var['name'] == "DMS_WAS":
        mask1 = (data[var["model_name"]] * var["factor"]) > .01
        data = data[mask1]
    
    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)



C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [63]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\winter"

os.makedirs(save_path, exist_ok=True)

campaign = "WINTER"
name_of_model = "GEOS-Chem" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "model_name": "GEOSChem_O3", "factor": 1e9, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "model_name": "GEOSChem_CO", "factor": 1e9, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "GEOSChem_CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    # {"name": "Ethane_WAS", "model_name": "GEOSChem_C2H6", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS_WAS", "model_name": "GEOSChem_DMS", "factor": 1e12, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = winter.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]

    mask = data[var["name"]] > 0

    data = data[mask]
    
    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]

    residuals = obs - model

    # Scatter plot: Observed vs. Residuals
    plt.figure(figsize=(8, 6))
    plt.scatter(obs, residuals, alpha=0.7)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Observed CO (ppbv)")
    plt.ylabel("Residuals (Observed - Predicted)")
    plt.title("Residual Plot")
    plt.savefig(os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}_residuals.png"))
    plt.clf()
    # plt.show()

    # Histogram: Residual distribution
    plt.figure(figsize=(8, 6))
    plt.hist(residuals, bins=30, color='skyblue', edgecolor='black')
    plt.axvline(0, color='red', linestyle='--')
    plt.xlabel("Residuals")
    plt.ylabel("Frequency")
    plt.title("Residual Distribution")
    plt.savefig(os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}_residual_distribution.png"))
    plt.clf()
    # plt.show()

    # Calculate standardized residuals
    mean_residual = np.mean(residuals)
    std_residual = np.std(residuals)
    standardized_residuals = (residuals - mean_residual) / std_residual

    # Flag outliers (e.g., standardized residual > |3|)
    outlier_indices = np.where(np.abs(standardized_residuals) > 3)[0]

    # Print number and indices of outliers
    print(f"Number of outliers: {len(outlier_indices)}")
    print(f"Outlier indices: {outlier_indices}")

    obs_filtered = obs.drop(index=obs.index[outlier_indices])
    pred_filtered = model.drop(index=model.index[outlier_indices])

    # Recalculate metrics with filtered data
    r2_filtered = r2_score(obs_filtered, pred_filtered)
    evs_filtered = explained_variance_score(obs_filtered, pred_filtered)

    print(f"Updated R^2: {r2_filtered}")
    print(f"Updated EVS: {evs_filtered}")

    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs_filtered, pred_filtered, title, var["label"], var["unit"], False, file_path)



Number of outliers: 11
Outlier indices: [451 466 468 469 493 494 495 496 499 500 501]
Updated R^2: -0.16006676801900532
Updated EVS: 0.3337843638702036


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


Number of outliers: 8
Outlier indices: [ 28 100 140 272 273 282 293 303]
Updated R^2: -0.3328825741151953
Updated EVS: 0.2745867977716733


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


Number of outliers: 0
Outlier indices: []
Updated R^2: -213.2268380247809
Updated EVS: -86.91857024669149


...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [67]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\intexb_c130_geos-chem"

os.makedirs(save_path, exist_ok=True)

campaign = "INTEXB"
name_of_model = "GEOS-Chem" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3_obs", "model_name": "O3_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO_obs", "model_name": "CO_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = intexb_c130_geos_chem_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]
    
    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [68]:
for c in intexb_c130_raqms_f.columns:
    print(c)

iindex
Fractional_Day
STARTTIME
STOPTIME
UTC_obs
JDAY_obs
INDEX_obs
FLIGHT_obs
LOCAL_SUN_TIME_obs
LAT
LON
ALT
PRESSURE_obs
TEMPERATURE_obs
THETA_obs
O3COLUMN_obs
SZA_obs
WNS_obs
WND_obs
Geometric_(Radar)_Altitude_(APN-232)
Corrected_GPS_Altitude
Aircraft_Roll_Angle
Aircraft_Pitch_Angle
Aircraft_True_Heading_Angle
GPS_Ground_Speed
GPS_Track_Angle
GPS_Ground_Speed_Vector_East_Component
GPS_Ground_Speed_Vector_North_Component
Aircraft_True_Airspeed_Reference
Corrected_Dynamic_Pressure_Reference
Interior_Cabin_Static_Pressure
Ambient_Water_Vapor_Pressure_Reference
Total_Temperature_Reference
Potential_Temperature
Equivalent_Potential_Temperature
Dew_Point_Temperature_Reference
Relative_Humidity
Radiometric_Surface_Temperature
Radiometric_Sky/Cloud-Base_Temperature
GPS-Corrected_Wind_Vector_East_Component
GPS-Corrected_Wind_Vector_North_Component
GPS-Corrected_Wind_Vector_Vertical_Gust_Component
Aircraft_Mach_Number_Squared
Shortwave_Irradiance_Top
Shortwave_Irradiance_Bottom
Ultraviolet_Ir

In [96]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\intexb_c130_raqms"

os.makedirs(save_path, exist_ok=True)

campaign = "INTEXB"
name_of_model = "RAQMS" 

# Define the metadata for each variable
variables = [
    {"name": "CH4", "model_name": "CH4_RAQMS", "factor": 1e6, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "model_name": "O3_RAQMS", "factor": 1e9, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "model_name": "CO_RAQMS", "factor": 1e9, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6_RAQMS", "factor": 1e12, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    # {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = intexb_c130_raqms_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]

    mask = data[var["model_name"]] > 0
    data = data[mask]

    if var['name'] == "O3":
        mask1 = (data[var["model_name"]] * var["factor"]) < 1500
        data = data[mask1]
    
    # Extract observed and model
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [122]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\intexb_dc8_geos-chem"

os.makedirs(save_path, exist_ok=True)

campaign = "INTEXB"
name_of_model = "GEOS-Chem" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "GMI_CH4_GMI", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3_obs", "model_name": "O3_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO_obs", "model_name": "CO_geos-chem", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = intexb_dc8_geos_chem_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]
    
    if var['model_name'] == "DMS":
        mask1 = ((data[var["model_name"]] * var["factor"]) > 1) & ((data[var["model_name"]] * var["factor"]) < 2000)
        data = data[mask1]

    # Extract observed and CESM data
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [143]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\intexb_dc8_raqms"

os.makedirs(save_path, exist_ok=True)

campaign = "INTEXB"
name_of_model = "RAQMS" 

# Define the metadata for each variable
variables = [
    {"name": "CH4", "model_name": "CH4_RAQMS", "factor": 1e9, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "model_name": "O3_RAQMS", "factor": 1e9, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "model_name": "CO_RAQMS", "factor": 1e9, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1e3, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    {"name": "Ethane_WAS", "model_name": "C2H6_RAQMS", "factor": 1e12, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    # {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = intexb_dc8_raqms_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    # most_frequent = data[var['model_name']].value_counts().idxmax()
    # data = data[data[var['model_name']] != most_frequent]

    mask = data[var["model_name"]] > 0
    data = data[mask]

    if var['name'] == "O3":
        mask1 = (data[var["model_name"]] * var["factor"]) < 10000
        data = data[mask1]
    
    # Extract observed and model
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)


C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\1440915340.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [127]:
for c in pta_aer_point_model_dc8_f.columns:
    print(c)

iindex
JDAY
STARTTIME
STOPTIME
MIDTIME
LAT
wLAT
LON
wLON
PSMB
wPSMB
ALT
wMALTP
MALTR
wMALTR
TS
wTS
TD
wTD
PITCH
wPITCH
ROLL
wROLL
MTAS
wMTAS
MGS
wMGS
HEAD
wHEAD
MACH
wMACH
DRIFT
wDRIFT
SURTEMPK
wSURTEMPK
calcRHw
wcalcRHw
calcRHi
wcalcRHi
calcQw
wcalcQw
calcQi
wcalcQi
calcPOTTEMP
wcalcPOTTEMP
WNS
wMWNS
WND
wWND
SUNELEG
wSUNELEG
SUNAZMG
wSUNAZMG
JNO2_Z
wJNO2_Z
JNO2_N
wJNO2_N
O3CLM
wO3CLM
TROPHEIGHT
wTROPHEIGHT
TROPO3COL
wTROPO3COL
O3COLbelow
wO3COLbelow
UF_AEROSOL
wUF_AEROSOL
F_AER_UNH
wF_AER_UNH
F_AER_H
wF_AER_H
ARATIO
wARATIO
CH3ONO2_A
wCH3ONO2_A
C2H5ONO2_A
wC2H5ONO2_A
i-C3H7ONO2_A
wi-C3H7ONO2_A
n-C3H7ONO2_A
wn-C3H7ONO2_A
2-C4H9ONO2_A
w2-C4H9ONO2_A
3-MC4H9ONO2_A
w3-MC4H9ONO2_A
3-C5H11ONO2_A
w3-C5H11ONO2_A
2-C5H11ONO2_A
w2-C5H11ONO2_A
CH2Cl2_A
wCH2Cl2_A
HCFC141B_A
wHCFC141B_A
HCFC142B_A
wHCFC142B_A
HFC134A_A
wHFC134A_A
CH2Br2_A
wCH2Br2_A
CHBrCl2_A
wCHBrCl2_A
CHBr2Cl_A
wCHBr2Cl_A
CHBr3_A
wCHBr3_A
SO2
wSO2
DMS_WAS
wDMS
H-1301_B
wH-1301_B
H-1211_B
wH-1211_B
H-2402_B
wH-2402_B
HCFC-22_B
wHC

In [140]:
test = pta_aer_point_model_dc8_f["O3_AER-point-model"]
A = test.iloc[:, 0]

A.value_counts()

O3_AER-point-model
5.200000e-08    2340
Name: count, dtype: int64

In [145]:
# Needs new join
# save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\pta_aer_dc8"

# os.makedirs(save_path, exist_ok=True)

# campaign = "PEM-Tropics-A"
# name_of_model = "AER-Point-Model" 

# # Define the metadata for each variable
# variables = [
#     {"name": "CH4_obs", "model_name": "CH4_AER-point-model", "factor": 1e9, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
#     # {"name": "O3_obs", "model_name": "O3_AER-point-model", "factor": 1e9, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
#     {"name": "CO_obs", "model_name": "CO_AER-point-model", "factor": 1e9, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
#     # {"name": "CH3Br_WAS", "model_name": "CH3BR", "factor": 1e12, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
#     {"name": "Ethane_WAS", "model_name": "C2H6", "factor": 1e12, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
#     # {"name": "DMS_WAS", "model_name": "DMS", "factor": 1e3, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
# ]

# # Automate the processing and plotting
# for var in variables:
#     # Filter the data for non-missing values
#     data = pta_aer_point_model_dc8_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

#     # most_frequent = data[var['model_name']].value_counts().idxmax()
#     # data = data[data[var['model_name']] != most_frequent]

#     mask = data[var["model_name"]] > 0
#     data = data[mask]

#     if var['name'] == "O3":
#         mask1 = (data[var["model_name"]] * var["factor"]) < 10000
#         data = data[mask1]
    
#     # Extract observed and model
#     obs = data[var["name"]]
#     model = data[var["model_name"]] * var["factor"]
    
#     # Generate the title and file path
#     title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
#     file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}.png")
    
#     # Call the scatter_plot function
#     scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)

# pta_aer_point_model_dc8_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\pta_aer_point_dc8_raqms.csv", index=False)

In [146]:
for c in korusaq_micp_f.columns:
    print(c)

iindex
JDAY (days)
STARTTIME (s)
STOPTIME (s)
UTC (s)
INDEX (none)
FLIGHT (none)
LOCAL_SUN_TIME (hour)
LAT
LON
ALT
PRESSURE (hPa)
TEMPERATURE (DegK)
THETA (K)
O3COLUMN (DU)
SZA (deg)
H2O_MixingRatio (ppmv)
ProjDP (K)
O3
CO
CH4
NO_10s_avg  (Mixing Ratio) (pptv)
NOy_10s_avg (Mixing Ratio) (pptv)
NO2 (Mixing Ratio) (pptv)
[OH] ((molecule/cc))
[H2SO4] ((molecule/cc))
[MSA] ((molecule/cc))
HNO3 (pptv)
RO2+HO2 (molecules-cm^-3)
HO2 (molecules-cm^03)
SO2 (pptv)
J[O3->O2+O(1D)] (1/s)
J[NO2->NO+O(3P)] (1/s)
J[N2O5->NO3+NO+O(3P)] (1/s)
J[N2O5->NO3+NO2] (1/s)
J[H2O2->2OH] (1/s)
J[HNO2->OH+NO] (1/s)
J[HNO3->OH+NO2] (1/s)
J[HNO4->HO2+NO2] (1/s)
J[CH2O->H+HCO] (1/s)
J[CH2O->H2+CO] (1/s)
J[CH3CHO->CH3+HCO] (1/s)
J[CH3CHO->CH4+CO] (1/s)
J[C2H5CHO->C2H5+HCO] (1/s)
J[CHOCHO] (1/s)
J[CH3COCHO] (1/s)
J[CH3COCH3] (1/s)
J[CH3OOH->CH3O+OH] (1/s)
J[CH3ONO2->CH3O+NO2] (1/s)
J[PAN+hv->products] (1/s)
J[CH3CH2CH2CHO->C3H7+HCO] (1/s)
J[CH3CH2CH2CHO->C2H4+CH2CHOH] (1/s)
J[CH3COCH2CH3] (1/s)
J[CH3CH2ONO2+hv->Produc

In [230]:
data["dms_WRFChemIOWA"] * 1e12

48      0.0997
50      0.0997
51      0.0997
53      0.0997
54      0.0997
         ...  
2595    0.0980
2596    0.0979
2597    0.0980
2600    0.0983
2601    0.0991
Name: dms_WRFChemIOWA, Length: 2037, dtype: float64

In [236]:
save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\korusaq_micp"

os.makedirs(save_path, exist_ok=True)

campaign = "KORUSAQ"
name_of_model = "CMAQINHA" 

# Define the metadata for each variable
variables = [
    # {"name": "CH4", "model_name": "CH4_RAQMS", "factor": 1, "unit": "ppbv", "file_suffix": "ch4", "label": "CH4"},
    {"name": "O3", "model_name": "vmro3_CMAQINHA", "factor": 1, "unit": "ppbv", "file_suffix": "o3", "label": "O3"},
    {"name": "CO", "model_name": "vmrco_CMAQINHA", "factor": 1, "unit": "ppbv", "file_suffix": "co", "label": "CO"},
    # {"name": "CH3Br_WAS", "model_name": "CH3Br", "factor": 1, "unit": "pptv", "file_suffix": "ch3br", "label": "CH3Br"},
    # {"name": "Ethane_WAS", "model_name": "c2h6_WRFChemUCLA", "factor": 1e3, "unit": "pptv", "file_suffix": "c2h6", "label": "Ethane"},
    # {"name": "DMS_WAS", "model_name": "dms_WRFChemIOWA", "factor": 1e12, "unit": "pptv", "file_suffix": "dms", "label": "DMS"}
]

# Automate the processing and plotting
for var in variables:
    # Filter the data for non-missing values
    data = korusaq_micp_f.dropna(subset=[var["name"], var["model_name"]]) ## CHANGE THE BIG DATFRAME HERE

    most_frequent = data[var['model_name']].value_counts().idxmax()
    data = data[data[var['model_name']] != most_frequent]

    mask = data[var["name"]] > 0
    data = data[mask]

    mask = data[var["model_name"]] > 0
    data = data[mask]

    # if var['name'] == "Ethane_WAS":
    #     mask1 = (data[var["model_name"]] * var["factor"]) < 12000
    #     data = data[mask1]
    
    # Extract observed and model
    obs = data[var["name"]]
    model = data[var["model_name"]] * var["factor"]
    
    # Generate the title and file path
    title = f'{name_of_model} Modeled {var["label"].lower()} for {campaign}'
    file_path = os.path.join(save_path, f"{campaign}_{name_of_model}_{var['file_suffix']}_linear.png")
    
    # Call the scatter_plot function
    scatter_plot(obs, model, title, var["label"], var["unit"], False, file_path)

# korusaq_micp_f.to_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\korusaq_micp.csv", index=False)

C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
C:\Users\vwgei\AppData\Local\Temp\ipykernel_6940\2037887402.py:57: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>